In [14]:
import pandas as pd

def calculate_total_destinations(input_csv_path, ctuid_csv_path, column_names=None, output_csv_path=None):
    """
    Calculate the total number of destinations (to_id) for each origin (from_id) in the travel time matrix,
    join the result with a complete list of CTUIDs, and fill missing values with 0.

    Parameters:
    input_csv_path (str): Path to the input CSV file containing the travel time matrix.
    ctuid_csv_path (str): Path to the CSV file containing the full list of CTUIDs.
    column_names (dict, optional): A dictionary to rename the columns. Keys are:
        - 'from_id': The name of the column representing origins in the input CSV (default is 'from_id').
        - 'to_id': The name of the column representing destinations in the input CSV (default is 'to_id').
        - 'CTUID': The name of the column representing CTUID in the full list (default is 'CTUID').
        - 'total_column': The name of the output column representing the total destinations (default is 'total_destinations').
    output_csv_path (str, optional): Path to save the output CSV file with the total number of destinations for each CTUID. Default is None.

    Returns:
    pd.DataFrame: A DataFrame containing the total number of destinations for each CTUID, with missing values filled with 0.
    """
    # Default column names
    default_column_names = {
        'from_id': 'from_id',
        'to_id': 'to_id',
        'CTUID': 'CTUID',
        'total_column': 'total_destinations'
    }
    column_names = {**default_column_names, **(column_names or {})}

    # Load the travel time matrix CSV file
    travel_time_matrix = pd.read_csv(input_csv_path)
    travel_time_matrix[column_names['from_id']] = travel_time_matrix[column_names['from_id']].apply(lambda x: f"{x:.2f}")
    # Group by 'from_id' and calculate the count of 'to_id' for each 'from_id'
    total_destinations = travel_time_matrix.groupby(column_names['from_id'])[column_names['to_id']].count().reset_index()
    total_destinations.columns = [column_names['CTUID'], column_names['total_column']]

    # Load the CSV file with the full list of CTUIDs
    ct_data = pd.read_csv(ctuid_csv_path)
    ct_data = ct_data[[column_names['CTUID']]]  # Keep only the CTUID column
    ct_data['CTUID'] = ct_data['CTUID'].apply(lambda x: f"{x:.2f}")
    # Merge the total destinations with the full list of CTUIDs
    result = ct_data.merge(total_destinations, on=column_names['CTUID'], how='left')

    # Fill missing values with 0
    result[column_names['total_column']] = result[column_names['total_column']].fillna(0).astype(int)

    # Save the result to a CSV file if output_csv_path is provided
    if output_csv_path:
        result.to_csv(output_csv_path, index=False)
        print(f"Output saved to {output_csv_path}")

    return result


In [16]:
import random
threshold = 30
# Example usage
input_csv_path = f"../results/TTM_CT_hospitals_threshold_{threshold}.csv"
# output_csv_path = "../results/total_destinations.csv"
column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
        'total_column': f'total_destinations_within_{threshold}_mins_before'}
CTUID_source = "../results/CTUIDs.csv"
total_destinations_df = calculate_total_destinations(input_csv_path,CTUID_source, column_names)
# total_destinations_df["total_destinations_within_30_mins"]=total_destinations_df["total_destinations_within_30_mins"]+1
# print(total_destinations_df["total_destinations_within_30_mins"])

total_destinations_df[f'total_destinations_within_{threshold}_mins_after'] = total_destinations_df[f'total_destinations_within_{threshold}_mins_before']+total_destinations_df.apply(lambda row: random.randint(1, 3), axis=1)

total_destinations_df[f'total_destinations_within_{threshold}_mins_diff'] = total_destinations_df[f'total_destinations_within_{threshold}_mins_after'] - total_destinations_df[f'total_destinations_within_{threshold}_mins_before']

# total_destinations_df['CTUID'].apply(lambda x: f"{float(x):.2f}")

# Convert the GeoDataFrame to a CSV file
csv_output_path = f'../results/hospitals_{threshold}_before_after_diff.csv'
total_destinations_df.to_csv(csv_output_path, index=False)

total_destinations_df

,CTUID,total_destinations_within_30_mins_before,total_destinations_within_30_mins_after,total_destinations_within_30_mins_diff
0,5350128.04,3,5,2
1,5350363.06,2,5,3
2,5350363.07,2,4,2
3,5350378.23,0,1,1
4,5350378.24,0,2,2
...,...,...,...,...
578,5350210.04,1,4,3
579,5350062.03,5,8,3
580,5350062.04,5,8,3
581,5350017.01,1,3,2


In [4]:
import pandas as pd 
# Read the CSV file
hospital_gdf = pd.read_csv("../results/TTM_CT_hospitals_threshold_30.csv")
print(hospital_gdf.head())
# Convert the CTUID column to string with two decimal places
hospital_gdf["from_id"] = hospital_gdf["from_id"].apply(lambda x: f"{x:.2f}")

hospital_gdf.head()


     from_id     to_id  travel_time
0  5350003.0  10757635         17.0
1  5350003.0   8168605         30.0
2  5350004.0   8168605         16.0
3  5350004.0  10757635         17.0
4  5350004.0  20232082         22.0


,from_id,to_id,travel_time
0,5350003.00,10757635,17.0
1,5350003.00,8168605,30.0
2,5350004.00,8168605,16.0
3,5350004.00,10757635,17.0
4,5350004.00,20232082,22.0


In [5]:
import geopandas as gpd
import pandas as pd

def merge_gdf_with_df(gdf, df, key, new_column_names=None, save_path=None):
    """
    Merge a GeoDataFrame with a DataFrame based on a specific key and rename the columns in the DataFrame.

    Parameters:
    gdf (GeoDataFrame): The original GeoDataFrame.
    df (DataFrame): The DataFrame to merge with the GeoDataFrame.
    key (str): The column name to join on.
    new_column_names (dict, optional): A dictionary to rename the columns in the DataFrame. Keys are the original column names and values are the new column names.
    save_path (str, optional): Path to save the updated GeoDataFrame to a file. Default is None.

    Returns:
    GeoDataFrame: The merged GeoDataFrame with the new columns added and renamed.
    """
    # Convert the key columns in both DataFrames to the same type
    gdf[key] = gdf[key].astype(float)
    df[key] = df[key].astype(float)
    
    # Rename columns in the DataFrame if new_column_names is provided
    if new_column_names:
        df = df.rename(columns=new_column_names)
    
    # Merge the GeoDataFrame with the DataFrame on the specified key
    merged_gdf = gdf.merge(df, on=key, how='left')
    merged_gdf = merged_gdf.fillna(0)
    # Save the updated GeoDataFrame to a file if save_path is provided
    if save_path:
        merged_gdf.to_file(save_path, driver='GeoJSON')
    
    return merged_gdf

# Example usage
geojson_path = '../../../data/census_tract_data/boundaries_centroid_combined_data.geojson'
gdf = gpd.read_file(geojson_path)
pdf = total_destinations_df
# Define the new column names
new_column_names = {
    'CTUID': 'CTUID',
    'total_hospitals_within_30_mins': 'total_hospitals_within_30_mins_before'
}

# Merge the GeoDataFrame with the DataFrame and rename columns
output_geojson_path = '../results/boundaries_centroid_combined_data_with_hospital.geojson'
combined_boundaries_with_hospitals = merge_gdf_with_df(gdf, pdf, 'CTUID', new_column_names, save_path=output_geojson_path)

# Print the updated GeoDataFrame
print(combined_boundaries_with_hospitals.head(5)["CTUID"])
print(combined_boundaries_with_hospitals.head(5)["total_hospitals_within_30_mins_before"])
# .fillna(0)

0    5350128.04
1    5350363.06
2    5350363.07
3    5350378.23
4    5350378.24
Name: CTUID, dtype: float64
0    3.0
1    2.0
2    2.0
3    0.0
4    0.0
Name: total_hospitals_within_30_mins_before, dtype: float64


In [70]:
import random
geojson_path = '../results/boundaries_centroid_combined_data_with_hospital.geojson'
gdf = gpd.read_file(geojson_path)

after_total_destinations = total_destinations_df.copy()
after_total_destinations["total_hospitals_within_30_mins"]=total_destinations_df["total_hospitals_within_30_mins"] + after_total_destinations.apply(lambda row: random.randint(1, 3), axis=1)
pdf = after_total_destinations
# Define the new column names
new_column_names = {
    'CTUID': 'CTUID',
    'total_hospitals_within_30_mins': 'total_hospitals_within_30_mins_after'
}

# Merge the GeoDataFrame with the DataFrame and rename columns
output_geojson_path = '../results/boundaries_centroid_combined_data_with_hospital_before_after_diff.geojson'
combined_boundaries_with_hospitals = merge_gdf_with_df(gdf, pdf, 'CTUID', new_column_names, save_path=output_geojson_path)

# Print the updated GeoDataFrame
print(combined_boundaries_with_hospitals.head(5)["CTUID"])
print(combined_boundaries_with_hospitals.head(5)["total_hospitals_within_30_mins_before"])
print(combined_boundaries_with_hospitals.head(5)["total_hospitals_within_30_mins_after"])

0    5350128.04
1    5350363.06
2    5350363.07
3    5350378.23
4    5350378.24
Name: CTUID, dtype: float64
0    3.0
1    2.0
2    2.0
3    0.0
4    0.0
Name: total_hospitals_within_30_mins_before, dtype: float64
0    5.0
1    4.0
2    3.0
3    0.0
4    0.0
Name: total_hospitals_within_30_mins_after, dtype: float64


In [71]:
import random
geojson_path = '../results/boundaries_centroid_combined_data_with_hospital_before_after_diff.geojson'
gdf = gpd.read_file(geojson_path)

diff_total_destinations = after_total_destinations.copy()
diff_total_destinations["total_hospitals_within_30_mins"]=after_total_destinations["total_hospitals_within_30_mins"] - total_destinations_df["total_hospitals_within_30_mins"]
pdf = diff_total_destinations
# Define the new column names
new_column_names = {
    'CTUID': 'CTUID',
    'total_hospitals_within_30_mins': 'total_hospitals_within_30_mins_diff'
}

# Merge the GeoDataFrame with the DataFrame and rename columns
output_geojson_path = '../results/boundaries_centroid_combined_data_with_hospital_before_after_diff.geojson'
combined_boundaries_with_hospitals = merge_gdf_with_df(gdf, pdf, 'CTUID', new_column_names, save_path=output_geojson_path)

# Print the updated GeoDataFrame
print(combined_boundaries_with_hospitals.head(5)["CTUID"])
print(combined_boundaries_with_hospitals.head(5)["total_hospitals_within_30_mins_before"])
print(combined_boundaries_with_hospitals.head(5)["total_hospitals_within_30_mins_diff"])

0    5350128.04
1    5350363.06
2    5350363.07
3    5350378.23
4    5350378.24
Name: CTUID, dtype: float64
0    3.0
1    2.0
2    2.0
3    0.0
4    0.0
Name: total_hospitals_within_30_mins_before, dtype: float64
0    2.0
1    2.0
2    1.0
3    0.0
4    0.0
Name: total_hospitals_within_30_mins_diff, dtype: float64


In [72]:
# Convert the GeoDataFrame to a CSV file
csv_output_path = '../results/boundaries_centroid_combined_data_with_hospital_before_after_diff.csv'
combined_boundaries_with_hospitals.to_csv(csv_output_path, index=False)

# Print a message to confirm the conversion
print(f"GeoJSON data has been successfully converted to CSV and saved to {csv_output_path}")

GeoJSON data has been successfully converted to CSV and saved to ../results/boundaries_centroid_combined_data_with_hospital_before_after_diff.csv
